# 12. UNDERSTANDING THE AGGREGATION FRAMEWORK
(_1h32min_)

## 12.1 Module Introduction

Retrieving data efficiently & in a structured way
1. [Aggregation Operations](https://www.mongodb.com/docs/manual/aggregation/#aggregation-operations)

---

## 12.2 What is the "Aggregation Framework"?

1. [Aggregation Stages](https://www.mongodb.com/docs/manual/reference/operator/aggregation-pipeline/#aggregation-stages)

> Aggregation operations process multiple documents and return computed results. You can use aggregation operations to:
>
> - Group values from multiple documents together.
> - Perform operations on the grouped data to return a single result.
> - Analyze data changes over time.
>
> To perform aggregation operations, you can use:
>
> - [Aggregation pipelines](https://www.mongodb.com/docs/manual/aggregation/#std-label-aggregation-pipeline-intro), which are the preferred method for performing aggregations.
> - [Single purpose aggregation methods](https://www.mongodb.com/docs/manual/aggregation/#std-label-single-purpose-agg-methods), which are simple but lack the capabilities of an aggregation pipeline.

<img src="imgs\s12\s12-1.png" width=400 height=300 >

---

## 12.3 Getting started with the aggregation pipeline

1. [Aggregation Pipeline](https://www.mongodb.com/docs/manual/core/aggregation-pipeline/#aggregation-pipeline)

- Importar o arquivo persons.json disponível na pasta "resources" do curso

In [ ]:
# C:\Projetos\mongodb-content\resources>
mongoimport persons.json -d analytics -c persons --jsonArray --drop

- Resultado da query `db.persons.findOne()`:

In [ ]:
{
  _id: ObjectId('67f9240d9da55dd301491eb8'),
  gender: 'male',
  name: { title: 'mr', first: 'carl', last: 'jacobs' },
  location: {
    street: '6948 springfield road',
    city: 'arklow',
    state: 'wicklow',
    postcode: 71309,
    coordinates: { latitude: '-29.6721', longitude: '-154.6037' },
    timezone: { offset: '-11:00', description: 'Midway Island, Samoa' }
  },
  email: 'carl.jacobs@example.com',
  login: {
    uuid: '4f591981-b214-4430-9902-70bc0faa7e81',
    username: 'organicladybug144',
    password: 'hank',
    salt: 'PC6Ig6sD',
    md5: 'd94aac977512cb2bb005dfa360b40018',
    sha1: 'a5ffeb65557693e443e195bdf9c066dca33dc47d',
    sha256: 'f9aa851b943d9a8a876062e48b91b9af190a37779df009a20bc268c25ce48a7f'
  },
  dob: { date: '1984-09-30T01:20:26Z', age: 33 },
  registered: { date: '2008-10-29T02:25:24Z', age: 9 },
  phone: '031-501-5147',
  cell: '081-090-3541',
  id: { name: 'PPS', value: '9806982T' },
  picture: {
    large: 'https://randomuser.me/api/portraits/men/44.jpg',
    medium: 'https://randomuser.me/api/portraits/med/men/44.jpg',
    thumbnail: 'https://randomuser.me/api/portraits/thumb/men/44.jpg'
  },
  nat: 'IE'
}

---

## 12.4 Using the aggregation framework

1. [`db.collection.aggregate()`](https://www.mongodb.com/docs/manual/reference/method/db.collection.aggregate/#db.collection.aggregate--)
2. [`$match` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/match/#-match--aggregation-)

> `db.collection.aggregate(pipeline, options)`
>
> Calculates aggregate values for the data in a collection or a view.
>
> - Returns:	
>
>   - A **cursor** for the documents produced by the final stage of the aggregation pipeline.
>
>   - If the pipeline includes the explain option, the query returns a document that provides details on the processing of the aggregation operation.
>
>   - If the pipeline includes the `$out` or `$merge` operators, the query returns an empty cursor.

In [ ]:
db.persons.aggregate([ 
    { $match: { gender: "female" } }
 ])

---

## 12.5 Understanding the Group Stage

1. [`$group (aggregation)`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/group/#-group--aggregation-)

> `$group`
>
> The `$group` stage combines **multiple documents with the same field**, fields, or expression into a single document according to a group key. The result is **one document per unique group key**.
>
> A group key is often a field, or group of fields. The group key can also be the result of an expression. Use the `_id` field in the `$group` pipeline stage to set the group key.
>
> In the `$group` stage output, the `_id` field is set to the group key for that document.


| **Field** | **Description**                                                                                                                                                                                                                                                                                                                                                             |
|:---------:| --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `_id`     | *Required.* The `_id` expression specifies the group key.<br/>If you specify an `_id` value of null, or any other constant value, the `$group` stage returns a single document that aggregates values across all of the input documents.<br/>[See the Group by Null example.](https://www.mongodb.com/docs/manual/reference/operator/aggregation/group/#std-label-null-example) |
| `field`   | *Optional.* Computed using the [accumulator operators.](https://www.mongodb.com/docs/manual/reference/operator/aggregation/group/#std-label-accumulators-group)                                                                                                                                                                                                             |

- Conteúdo do arquivo disponível na pasta "resources" -> `commands-1.js`:

In [ ]:
db.persons.aggregate([
    { $match: { gender: 'female' } },
    { $group: { _id: { state: "$location.state" }, totalPersons: { $sum: 1 } } }
])

- Output da query acima (somente uma amostra, o retorno em quantidade é maior):
    - para cada estado, é retornado a quantidade de gêneros "female" existentes
    - por exemplo: em "Ohio" existe somente 1 documento com gênero "female" na collection

In [ ]:
[
    { _id: { state: 'glarus' }, totalPersons: 4 },
    { _id: { state: 'basel-stadt' }, totalPersons: 6 },
    { _id: { state: 'ohio' }, totalPersons: 1 },
    { _id: { state: 'aydın' }, totalPersons: 3 },
    { _id: { state: 'haute-corse' }, totalPersons: 3 },
    { _id: { state: 'paraná' }, totalPersons: 3 },
    { _id: { state: 'rheinland-pfalz' }, totalPersons: 8 },
    { _id: { state: 'kansas' }, totalPersons: 2 },
    { _id: { state: 'thurgau' }, totalPersons: 5 },
    { _id: { state: 'cantal' }, totalPersons: 4 },
    { _id: { state: 'کهگیلویه و بویراحمد' }, totalPersons: 8 },
    { _id: { state: 'acre' }, totalPersons: 2 },
    { _id: { state: 'puy-de-dôme' }, totalPersons: 1 },
    { _id: { state: 'midtjylland' }, totalPersons: 33 },
    { _id: { state: 'hawaii' }, totalPersons: 3 },
    { _id: { state: 'hérault' }, totalPersons: 2 },
    { _id: { state: 'ontario' }, totalPersons: 12 },
    { _id: { state: 'mardin' }, totalPersons: 3 },
    { _id: { state: 'burdur' }, totalPersons: 2 },
    { _id: { state: 'corrèze' }, totalPersons: 1 }
  ]

- Ao executar a query abaixo, 2 documentos são retornados, e somente 1 deles tem o gênero como "female":

In [ ]:
db.persons.find( {"location.state": "ohio"})

---

## 12.6 Diving deeper into the Group Stage

1. [`$sort` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/sort/#-sort--aggregation-)

> `$sort`
>
> Sorts all input documents and returns them to the pipeline in sorted order.

- A query abaixo faz uma busca em ordem decrescente da quantidade de pessoas com gênero "female" que vivem em cada "state"

In [ ]:
db.persons.aggregate([
    { $match: { gender: 'female' } },
    { $group: { _id: { state: "$location.state" }, totalPersons: { $sum: 1 } } },
    { $sort: { totalPersons: -1 }}
])

---

## 12.7 Assignment: Time to Practice - The Aggregation Framework

1. [`$avg` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/avg/#-avg--aggregation-)

### 12.7.1 Activity

- Build a pipeline to find only persons older than 50
- group than by gender
- find out how many persons we have by gender 
- the average age by gender


In [ ]:
db.persons.aggregate( [ 
    { $match: { "dob.age": { $gt: 50 } }},
    { $group: { _id: { gender: "$gender" }, totalPersons: { $sum: 1 }, avgAge: { $avg: "$dob.age" } } },
    { $sort: { totalPersons: -1 } }
])

- Output da query acima:

In [ ]:
[
    {
      _id: { gender: 'female' },
      totalPersons: 1125,
      avgAge: 61.90577777777778
    },
    {
      _id: { gender: 'male' },
      totalPersons: 1079,
      avgAge: 62.066728452270624
    }
  ]

---

## 12.8 Working with `$project`

1. [`$project` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/project/#-project--aggregation-)
2. [`$concat` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/concat/#-concat--aggregation-)
3. [Expression Operators](https://www.mongodb.com/docs/manual/reference/operator/aggregation/#expression-operators)

>`$project`
>
> Passes along the documents with the requested fields to the next stage in the pipeline. The specified fields can be existing fields from the input documents or newly computed fields.

In [ ]:
db.persons.aggregate( [
    { $project: 
        { _id: 0,
          gender: 1,
          fullName: { $concat: [ "$name.first", " ", "$name.last" ]}
        }
    }, { $sort: { fullName: 1 } }
])

- Output parcial da query acima:

In [ ]:
[
    { gender: 'female', fullName: 'aada hakola' },
    { gender: 'female', fullName: 'aada lepisto' },
    { gender: 'female', fullName: 'aada seppala' },
    { gender: 'female', fullName: 'aada waisanen' },
    { gender: 'female', fullName: 'aada walli' },
    { gender: 'female', fullName: 'aafke cameron' },
    { gender: 'male', fullName: 'aapo tuominen' },
    { gender: 'male', fullName: 'aaron bertrand' },
    { gender: 'male', fullName: 'aaron cooper' },
    { gender: 'male', fullName: 'aaron duncan' },
    { gender: 'male', fullName: 'aaron duval' },
    { gender: 'male', fullName: 'aaron edwards' },
    { gender: 'male', fullName: 'aatu waara' },
    { gender: 'female', fullName: 'aaya den os' },
    { gender: 'female', fullName: 'abbey williams' },
    { gender: 'female', fullName: 'abbie brown' },
    { gender: 'female', fullName: 'abby fernandez' },
    { gender: 'male', fullName: 'abdelhakim leemburg' },
    { gender: 'male', fullName: 'abdullahi fjelldal' },
    { gender: 'female', fullName: 'abigail brady' }
  ]

- É possível realizar outras modificações com o `$project`, como alterar as palavras para _uppercase_:

> `$toUpper`: Converts a string to uppercase. Accepts a single argument expression.

In [ ]:
db.persons.aggregate( [
    { $project: 
        { _id: 0,
          gender: 1,
          fullName: { $concat: [ { $toUpper: "$name.first" }, " ", { $toUpper: "$name.last" } ]}
        }
    }, { $sort: { fullName: 1 } }
])

- Output parcial da query acima:

In [ ]:
[
    { gender: 'female', fullName: 'AADA HAKOLA' },
    { gender: 'female', fullName: 'AADA LEPISTO' },
    { gender: 'female', fullName: 'AADA SEPPALA' },
    { gender: 'female', fullName: 'AADA WAISANEN' },
    { gender: 'female', fullName: 'AADA WALLI' },
    { gender: 'female', fullName: 'AAFKE CAMERON' },
    { gender: 'male', fullName: 'AAPO TUOMINEN' },
    { gender: 'male', fullName: 'AARON BERTRAND' },
    { gender: 'male', fullName: 'AARON COOPER' },
    { gender: 'male', fullName: 'AARON DUNCAN' },
    { gender: 'male', fullName: 'AARON DUVAL' },
    { gender: 'male', fullName: 'AARON EDWARDS' },
    { gender: 'male', fullName: 'AATU WAARA' },
    { gender: 'female', fullName: 'AAYA DEN OS' },
    { gender: 'female', fullName: 'ABBEY WILLIAMS' },
    { gender: 'female', fullName: 'ABBIE BROWN' },
    { gender: 'female', fullName: 'ABBY FERNANDEZ' },
    { gender: 'male', fullName: 'ABDELHAKIM LEEMBURG' },
    { gender: 'male', fullName: 'ABDULLAHI FJELLDAL' },
    { gender: 'female', fullName: 'ABIGAIL BRADY' }
  ]

- Também é possível trabalhar com alterações somente em alguns trechos da string
- O exemplo da query abaixo transforma somente a primeira letra dos nomes em maiúscula

> - `$substrCP`: Returns the **substring of a string**. Starts with the character at the specified UTF-8 code point (CP) index (zero-based) in the string and continues for the number of code points specified.
>
> - `$strLenCP`: Returns the **number of UTF-8 code points in a string**.
>
> - `$subtract`: Returns the result of **subtracting the second value from the first**. If the two values are numbers, return the difference. If the two values are dates, return the difference in milliseconds. If the two values are a date and a number in milliseconds, return the resulting date. Accepts two argument expressions. If the two values are a date and a number, specify the date argument first as it is not meaningful to subtract a date from a number.

In [ ]:
db.persons.aggregate( [
    { $project: {
        _id: 0,
        gender: 1,
        fullName: {
          $concat: [
            { $toUpper: { $substrCP: [ "$name.first", 0, 1] } },
            { $substrCP: [ "$name.first", 1, { $subtract: [ { $strLenCP: "$name.first"}, 1 ] } ] },
            " ",
            { $toUpper: { $substrCP: [ "$name.last", 0, 1] } },
            { $substrCP: [ "$name.last", 1, { $subtract: [ { $strLenCP: "$name.last"}, 1 ] } ] }
        ] }
    } },
    { $sort: { fullName: 1 } }
])

- Output parcial da query acima:

In [ ]:
[
    { gender: 'female', fullName: 'Aada Hakola' },
    { gender: 'female', fullName: 'Aada Lepisto' },
    { gender: 'female', fullName: 'Aada Seppala' },
    { gender: 'female', fullName: 'Aada Waisanen' },
    { gender: 'female', fullName: 'Aada Walli' },
    { gender: 'female', fullName: 'Aafke Cameron' },
    { gender: 'male', fullName: 'Aapo Tuominen' },
    { gender: 'male', fullName: 'Aaron Bertrand' },
    { gender: 'male', fullName: 'Aaron Cooper' },
    { gender: 'male', fullName: 'Aaron Duncan' },
    { gender: 'male', fullName: 'Aaron Duval' },
    { gender: 'male', fullName: 'Aaron Edwards' },
    { gender: 'male', fullName: 'Aatu Waara' },
    { gender: 'female', fullName: 'Aaya Den os' },
    { gender: 'female', fullName: 'Abbey Williams' },
    { gender: 'female', fullName: 'Abbie Brown' },
    { gender: 'female', fullName: 'Abby Fernandez' },
    { gender: 'male', fullName: 'Abdelhakim Leemburg' },
    { gender: 'male', fullName: 'Abdullahi Fjelldal' },
    { gender: 'female', fullName: 'Abigail Brady' }
  ]

---

## 12.9 Turning the location into a geoJSON object

1. [`$convert` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/convert/#-convert--aggregation-)

In [ ]:
db.persons.aggregate( [
    {
        $project: {
            _id: 0, 
            name: 1, 
            email: 1,
            location: { 
                type: "Point",
                coordinates: [ 
                    "$location.coordinates.longitude", 
                    "$location.coordinates.latitude"
                ]
            }
        }
    },
    { $project: {
        gender: 1,
        email: 1,
        location: 1,
        fullName: {
          $concat: [
            { $toUpper: { $substrCP: [ "$name.first", 0, 1] } },
            { $substrCP: [ "$name.first", 1, { $subtract: [ { $strLenCP: "$name.first"}, 1 ] } ] },
            " ",
            { $toUpper: { $substrCP: [ "$name.last", 0, 1] } },
            { $substrCP: [ "$name.last", 1, { $subtract: [ { $strLenCP: "$name.last"}, 1 ] } ] }
        ] }
    } },
    { $sort: { fullName: 1 } }
])

- Output da query acima:
    - note que o campo das coordenadas é uma string
    - precisaremos converter isto

In [ ]:
[
  {
    location: { type: 'Point', coordinates: [ '-179.7521', '30.6068' ] },
    email: 'abdullahi.fjelldal@example.com',
    fullName: 'Abdullahi Fjelldal'
  },
  {
    location: { type: 'Point', coordinates: [ '-152.6688', '-28.8985' ] },
    email: 'abigail.brady@example.com',
    fullName: 'Abigail Brady'
  }
]

- Usando a expressão `$convert`, podemos converter o campo para outro tipo de dado

> `$convert`: Converts a value to a specified type.

In [ ]:
db.persons.aggregate( [
    {
        $project: { _id: 0, name: 1, email: 1, location:
            { 
                type: "Point",
                coordinates: [ 
                    { $convert: { input: "$location.coordinates.longitude", to: "double", onError: 0.0, onNull: 0.0 } }, 
                    { $convert: { input: "$location.coordinates.latitude", to: "double", onError: 0.0, onNull: 0.0 } }
                ]
            }
        }
    },
    { 
        $project: { gender: 1, email: 1, location: 1, fullName:
            {
                $concat: [
                    { $toUpper: { $substrCP: [ "$name.first", 0, 1] } },
                    { $substrCP: [ "$name.first", 1, { $subtract: [ { $strLenCP: "$name.first"}, 1 ] } ] },
                    " ",
                    { $toUpper: { $substrCP: [ "$name.last", 0, 1] } },
                    { $substrCP: [ "$name.last", 1, { $subtract: [ { $strLenCP: "$name.last"}, 1 ] } ] }
                ]
            }
        }
    }, { $sort: { fullName: 1 } } ] )

- Output da query acima:
    - note que o campo das coordenadas agora é numérico

In [ ]:
[
  {
    location: { type: 'Point', coordinates: [ -179.7521, 30.6068 ] },
    email: 'abdullahi.fjelldal@example.com',
    fullName: 'Abdullahi Fjelldal'
  },
  {
    location: { type: 'Point', coordinates: [ -152.6688, -28.8985 ] },
    email: 'abigail.brady@example.com',
    fullName: 'Abigail Brady'
  }
]

---

## 12.10 Transforming the Birthdate

- A ideia agora é transformar a informação do 'nested document' `dob` em campos principais:
 - birthdate: com a data de nascimento da pessoa
 - age: com a idade da pessoa
    - o campo birthdate terá que ser convertido para o formato "date"

In [ ]:
db.persons.aggregate( [
    {
        $project: { _id: 0, name: 1, email: 1,
            birthdate:
            {
                $convert: { input: "$dob.date", to: "date" }
            },
            age: "$dob.age",
            location:
            { 
                type: "Point",
                coordinates: [ 
                    { $convert: { input: "$location.coordinates.longitude", to: "double", onError: 0.0, onNull: 0.0 } }, 
                    { $convert: { input: "$location.coordinates.latitude", to: "double", onError: 0.0, onNull: 0.0 } }
                ]
            }
        }
    },
    { 
        $project: { gender: 1, email: 1, location: 1, birthdate: 1, age: 1, fullName:
            {
                $concat: [
                    { $toUpper: { $substrCP: [ "$name.first", 0, 1] } },
                    { $substrCP: [ "$name.first", 1, { $subtract: [ { $strLenCP: "$name.first"}, 1 ] } ] },
                    " ",
                    { $toUpper: { $substrCP: [ "$name.last", 0, 1] } },
                    { $substrCP: [ "$name.last", 1, { $subtract: [ { $strLenCP: "$name.last"}, 1 ] } ] }
                ]
            }
        }
    }, { $sort: { fullName: 1 } } ] )

- Output da query acima:

In [ ]:
[
  {
    location: { type: 'Point', coordinates: [ -179.7521, 30.6068 ] },
    email: 'abdullahi.fjelldal@example.com',
    birthdate: ISODate('1966-11-29T22:28:10.000Z'),
    age: 51,
    fullName: 'Abdullahi Fjelldal'
  },
  {
    location: { type: 'Point', coordinates: [ -152.6688, -28.8985 ] },
    email: 'abigail.brady@example.com',
    birthdate: ISODate('1968-09-20T05:32:57.000Z'),
    age: 49,
    fullName: 'Abigail Brady'
  }
]

---

## 12.11 Using shortcuts for transformations

1. [`$toDate` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/toDate/#-todate--aggregation-)

> In addition to `$convert`, MongoDB provides the following aggregation operators as shorthand when the default "**onError**" and "**onNull**" behavior is acceptable:
>
> `$toBool`
>
> `$toDate`
>
> `$toDecimal`
>
> `$toDouble`
>
> `$toInt`
>
> `$toLong`
>
> `$toObjectId`
>
> `$toString`
>
> `$toUUID`

- Como não usamos "onError" ou "onNull" no campo `birthdate`, vamos reescrever a query usando um _shortcut_:

In [ ]:
db.persons.aggregate( [
    {
        $project: { _id: 0, name: 1, email: 1,
            birthdate:
            {
                $toDate: "$dob.date"
            },
            age: "$dob.age",
            location:
            { 
                type: "Point",
                coordinates: [ 
                    { $convert: { input: "$location.coordinates.longitude", to: "double", onError: 0.0, onNull: 0.0 } }, 
                    { $convert: { input: "$location.coordinates.latitude", to: "double", onError: 0.0, onNull: 0.0 } }
                ]
            }
        }
    }
])

- Output da query acima:

In [ ]:
[
  {
    name: { title: 'mr', first: 'gideon', last: 'van drongelen' },
    location: { type: 'Point', coordinates: [ -54.1364, -86.1268 ] },
    email: 'gideon.vandrongelen@example.com',
    birthdate: ISODate('1971-03-28T04:47:21.000Z'),
    age: 47
  },
  {
    name: { title: 'mademoiselle', first: 'delia', last: 'durand' },
    location: { type: 'Point', coordinates: [ -90.4049, -65.0877 ] },
    email: 'delia.durand@example.com',
    birthdate: ISODate('1966-08-03T09:22:41.000Z'),
    age: 52
  }
]

---

## 12.12 Understanding the `$isoWeekYear` operator

1. [`$isoWeekYear` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/isoWeekYear/#-isoweekyear--aggregation-)

> `$isoWeekYear`: Returns the year number in ISO 8601 format. The year starts with the Monday of week 1 and ends with the Sunday of the last week.

| **Field**  | **Description**                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
|:----------:| -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `date`     | The date to which the operator is applied. `<dateExpression>` must be a valid [expression](https://www.mongodb.com/docs/manual/reference/operator/aggregation/#std-label-aggregation-expressions) that resolves to a [Date](https://www.mongodb.com/docs/manual/reference/bson-types/#std-label-document-bson-type-date), a [Timestamp](https://www.mongodb.com/docs/manual/reference/bson-types/#std-label-document-bson-type-timestamp), or an [ObjectID.](https://www.mongodb.com/docs/manual/reference/bson-types/#std-label-document-bson-type-object-id) |
| `timezone` | Optional. The timezone of the operation result. `<tzExpression>` must be a valid [expression](https://www.mongodb.com/docs/manual/reference/operator/aggregation/#std-label-aggregation-expressions) that resolves to a string formatted as either an [Olson Timezone Identifier](https://en.wikipedia.org/wiki/List_of_tz_database_time_zones) or a [UTC Offset](https://en.wikipedia.org/wiki/List_of_UTC_time_offsets).<br>If no `timezone` is provided, the result is in UTC.                                                                                 |


- Vamos agrupar as pessoas pelo ano de nascimento e ordenar pelos anos com mais pessoas nascidas

In [ ]:
db.persons.aggregate( [
    {
        $project: { _id: 0, name: 1, email: 1,
            birthdate: { $toDate: "$dob.date" },
            age: "$dob.age",
            location:
            { 
                type: "Point",
                coordinates: [ 
                    { $convert: { input: "$location.coordinates.longitude", to: "double", onError: 0.0, onNull: 0.0 } }, 
                    { $convert: { input: "$location.coordinates.latitude", to: "double", onError: 0.0, onNull: 0.0 } }
                ]
            }
        }
    },
    { 
        $project: { gender: 1, email: 1, location: 1, birthdate: 1, age: 1, fullName:
            {
                $concat: [
                    { $toUpper: { $substrCP: [ "$name.first", 0, 1] } },
                    { $substrCP: [ "$name.first", 1, { $subtract: [ { $strLenCP: "$name.first"}, 1 ] } ] },
                    " ",
                    { $toUpper: { $substrCP: [ "$name.last", 0, 1] } },
                    { $substrCP: [ "$name.last", 1, { $subtract: [ { $strLenCP: "$name.last"}, 1 ] } ] }
                ]
            }
        }
    },
    { $group: { _id: { birthYear: { $isoWeekYear: "$birthdate" }}, numPersons: { $sum: 1} } },
    { $sort: { numPersons: -1 } }
] )

- Output parcial da query acima:

In [ ]:
[
  { _id: { birthYear: Long('1955') }, numPersons: 113 },
  { _id: { birthYear: Long('1961') }, numPersons: 111 },
  { _id: { birthYear: Long('1960') }, numPersons: 110 },
  { _id: { birthYear: Long('1993') }, numPersons: 110 },
  { _id: { birthYear: Long('1975') }, numPersons: 107 },
  { _id: { birthYear: Long('1945') }, numPersons: 106 },
  { _id: { birthYear: Long('1976') }, numPersons: 105 },
  { _id: { birthYear: Long('1967') }, numPersons: 104 },
  { _id: { birthYear: Long('1990') }, numPersons: 103 },
  { _id: { birthYear: Long('1981') }, numPersons: 102 },
  { _id: { birthYear: Long('1994') }, numPersons: 102 },
  { _id: { birthYear: Long('1958') }, numPersons: 101 },
  { _id: { birthYear: Long('1995') }, numPersons: 101 },
  { _id: { birthYear: Long('1948') }, numPersons: 100 },
  { _id: { birthYear: Long('1946') }, numPersons: 100 },
  { _id: { birthYear: Long('1983') }, numPersons: 99 },
  { _id: { birthYear: Long('1970') }, numPersons: 99 },
  { _id: { birthYear: Long('1950') }, numPersons: 99 },
  { _id: { birthYear: Long('1963') }, numPersons: 98 },
  { _id: { birthYear: Long('1965') }, numPersons: 98 }
]

---

## 12.13 $group vs. $project

- `$group`: is for grouping multiple documents into 1 document
- `$project`: is 1:1 relation -> 1 give 1 document and get 1 document back, but changed

<img src="imgs\s12\s12-2.png" width=700 height=350 >

---

## 12.14 Pushing elements to a newly created arrays

1. [`$push` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/push/#-push--aggregation-)

- o arquivo para criação da *collection* está em "resources" -> `array-data.json`

In [ ]:
db.friends.insertMany(
    [
        {
          "name": "Max",
          "hobbies": ["Sports", "Cooking"],
          "age": 29,
          "examScores": [
            { "difficulty": 4, "score": 57.9 },
            { "difficulty": 6, "score": 62.1 },
            { "difficulty": 3, "score": 88.5 }
          ]
        },
        {
          "name": "Manu",
          "hobbies": ["Eating", "Data Analytics"],
          "age": 30,
          "examScores": [
            { "difficulty": 7, "score": 52.1 },
            { "difficulty": 2, "score": 74.3 },
            { "difficulty": 5, "score": 53.1 }
          ]
        },
        {
          "name": "Maria",
          "hobbies": ["Cooking", "Skiing"],
          "age": 29,
          "examScores": [
            { "difficulty": 3, "score": 75.1 },
            { "difficulty": 8, "score": 44.2 },
            { "difficulty": 6, "score": 61.5 }
          ]
        }
    ]
)

- merge or combine arrays in a group stage:

In [ ]:
db.friends.aggregate( [
    { $group: { _id: { age: "$age"}, allHobbies: { $push: "$hobbies" } } }
])

- Output da query acima:

In [ ]:
[
    {
      _id: { age: 29 },
      allHobbies: [ [ 'Sports', 'Cooking' ], [ 'Cooking', 'Skiing' ] ]
    },
    { _id: { age: 30 }, allHobbies: [ [ 'Eating', 'Data Analytics' ] ] }
  ]

---

## 12.15 Understanding the `$unwind` stage

1. [`$unwind` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/unwind/#-unwind--aggregation-)

> `$unwind`:
>
> Deconstructs an array field from the input documents to output a document for each element.
>
> Each output document is the input document with the value of the array field replaced by the element.

### Exemplo 1:

In [ ]:
db.friends.aggregate( [
    { $unwind: "$hobbies" }
])

- Output da query acima:

In [ ]:
[
    {
      _id: ObjectId('67f96c55614df4be84b71236'),
      name: 'Max',
      hobbies: 'Sports',
      age: 29,
      examScores: [
        { difficulty: 4, score: 57.9 },
        { difficulty: 6, score: 62.1 },
        { difficulty: 3, score: 88.5 }
      ]
    },
    {
      _id: ObjectId('67f96c55614df4be84b71236'),
      name: 'Max',
      hobbies: 'Cooking',
      age: 29,
      examScores: [
        { difficulty: 4, score: 57.9 },
        { difficulty: 6, score: 62.1 },
        { difficulty: 3, score: 88.5 }
      ]
    },
    {
      _id: ObjectId('67f96c55614df4be84b71237'),
      name: 'Manu',
      hobbies: 'Eating',
      age: 30,
      examScores: [
        { difficulty: 7, score: 52.1 },
        { difficulty: 2, score: 74.3 },
        { difficulty: 5, score: 53.1 }
      ]
    },
    {
      _id: ObjectId('67f96c55614df4be84b71237'),
      name: 'Manu',
      hobbies: 'Data Analytics',
      age: 30,
      examScores: [
        { difficulty: 7, score: 52.1 },
        { difficulty: 2, score: 74.3 },
        { difficulty: 5, score: 53.1 }
      ]
    },
    {
      _id: ObjectId('67f96c55614df4be84b71238'),
      name: 'Maria',
      hobbies: 'Cooking',
      age: 29,
      examScores: [
        { difficulty: 3, score: 75.1 },
        { difficulty: 8, score: 44.2 },
        { difficulty: 6, score: 61.5 }
      ]
    },
    {
      _id: ObjectId('67f96c55614df4be84b71238'),
      name: 'Maria',
      hobbies: 'Skiing',
      age: 29,
      examScores: [
        { difficulty: 3, score: 75.1 },
        { difficulty: 8, score: 44.2 },
        { difficulty: 6, score: 61.5 }
      ]
    }
  ]

### Exemplo 2:

In [ ]:
db.friends.aggregate( [
    { $unwind: "$hobbies" },
    { $group: { _id: { age: "$age"}, allHobbies: { $push: "$hobbies" } } }
])

- Output da query acima:

In [ ]:
[
  {
    _id: { age: 29 },
    allHobbies: [ 'Sports', 'Cooking', 'Cooking', 'Skiing' ]
  },
  { _id: { age: 30 }, allHobbies: [ 'Eating', 'Data Analytics' ] }
]

---

## 12.16 Eliminating duplicated values

1. [`$addToSet`](https://www.mongodb.com/docs/manual/reference/operator/update/addToSet/#-addtoset)
2. [`$addToSet` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/addToSet/#-addtoset--aggregation-)

> `$addToSet` returns an array of all unique values that results from applying an expression to each document in a group.
>
> The order of the elements in the returned array is unspecified.

In [ ]:
db.friends.aggregate( [
    { $unwind: "$hobbies" },
    { $group: { _id: { age: "$age"}, allHobbies: { $addToSet: "$hobbies" } } }
])

- Output da query acima, evitando resultados duplicados:

In [ ]:
[
    { _id: { age: 29 }, allHobbies: [ 'Cooking', 'Skiing', 'Sports' ] },
    { _id: { age: 30 }, allHobbies: [ 'Eating', 'Data Analytics' ] }
  ]

---

## 12.16 Using Projection with Arrays

1. [`$slice` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/slice/#-slice--aggregation-)

- Digamos que queremos que seja exibido **somente o primeiro resultado** de um array:

In [ ]:
db.friends.aggregate([ { $project: { _id: 0, examScore: { $slice: ["$examScores", 1] }}} ])

- Output da query acima:

In [ ]:
[
    { examScore: [ { difficulty: 4, score: 57.9 } ] },
    { examScore: [ { difficulty: 7, score: 52.1 } ] },
    { examScore: [ { difficulty: 3, score: 75.1 } ] }
  ]

- Digamos que queremos que seja exibido **somente os últimos 2 resultados** de um array:

In [ ]:
db.friends.aggregate([ { $project: { _id: 0, examScore: { $slice: ["$examScores", -2] }}} ])

- Output da query acima:

In [ ]:
[
  {
    examScore: [ { difficulty: 6, score: 62.1 }, { difficulty: 3, score: 88.5 } ]
  },
  {
    examScore: [ { difficulty: 2, score: 74.3 }, { difficulty: 5, score: 53.1 } ]
  },
  {
    examScore: [ { difficulty: 8, score: 44.2 }, { difficulty: 6, score: 61.5 } ]
  }
]

- Digamos que queremos que seja exibido **somente 1 resultado** iniciando o array **à partir do 2º elemento**:

In [ ]:
db.friends.aggregate([ { $project: { _id: 0, examScore: { $slice: ["$examScores", 2, 1] }}} ])

---

## 2.17 Getting the length of an array

1. [`$size` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/size/#-size--aggregation-)

> `$size`: Counts and returns the total number of items in an array.

- Digamos que queremos saber a quantidade de valores dentro de um array:

In [ ]:
db.friends.aggregate([ { $project: { _id: 0, numScores: { $size: [ "$examScores" ] }}} ])

- Output da query acima:

In [ ]:
[ { numScores: 3 }, { numScores: 3 }, { numScores: 3 } ]

---

## 2.18 Using the "$filter" operator

1. [`$filter` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/filter/#-filter--aggregation-)

>`$filter`
>
> Selects a **subset of an array to return based on the specified condition**. Returns an array with **only those elements** that match the condition. The returned elements are in the original order.

- Vamos criar uma query onde só queremos resultados de **"examScores" com valor maior que 60**:

    - `"$sc"`: geraria uma comparação de strings
    - `"$$sc"`: gera uma comparação numérica

In [ ]:
db.friends.aggregate( [
    { $project: { 
        _id: 0, 
        scores: { 
            $filter: { 
                input: "$examScores", 
                as: "sc", 
                cond: { $gt: ["$$sc.score", 60] } 
            }
        }
    } }
] )

- Output da query acima:

In [ ]:
[
    {
      scores: [ { difficulty: 6, score: 62.1 }, { difficulty: 3, score: 88.5 } ]
    },
    { scores: [ { difficulty: 2, score: 74.3 } ] },
    {
      scores: [ { difficulty: 3, score: 75.1 }, { difficulty: 6, score: 61.5 } ]
    }
  ]

---

## 2.19 Applying multiple operations to our Array

1. [`$first` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/first/#-first--aggregation-)

- Exibir o maior "examScore" de cada pessoa da coleção:
    - unwind, sort and then group together by person and take the first score

> `$first`: Returns the result of an expression **for the first document in a group of documents**. Only meaningful when documents are in a defined order.

In [ ]:
db.friends.aggregate([
    { $unwind: "$examScores" },
    { $project: { _id: 1, name: 1, age: 1, score: "$examScores.score"} },
    { $sort: { score: -1} },
    { $group: { _id: "$_id", name: { $first: "$name" }, maxScore: { $max: "$score"} } },
    { $sort: { maxScore: -1}}
])

- Output da query acima:

In [ ]:
[
    {
      _id: ObjectId('67f96c55614df4be84b71236'),
      name: 'Max',
      maxScore: 88.5
    },
    {
      _id: ObjectId('67f96c55614df4be84b71238'),
      name: 'Maria',
      maxScore: 75.1
    },
    {
      _id: ObjectId('67f96c55614df4be84b71237'),
      name: 'Manu',
      maxScore: 74.3
    }
  ]

---

## 2.20 Understanding "$bucket"

1. [`$bucket` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#-bucket--aggregation-)

> `$bucket`:
> categorizes incoming documents into groups, called buckets, based on a specified expression and bucket boundaries and outputs a document per each bucket. Each output document contains an _id field whose value specifies the inclusive lower bound of the bucket. The output option specifies the fields included in each output document.
>
> $bucket only produces output documents for buckets that contain at least one input document.

| Field                                                                                                                | Type       | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
| -------------------------------------------------------------------------------------------------------------------- | ---------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| [groupBy](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#std-label-bucket-group-by)      | expression | An [expression](https://www.mongodb.com/docs/manual/reference/operator/aggregation/#std-label-aggregation-expressions) to group documents by.<br>To specify a [field path](https://www.mongodb.com/docs/manual/core/aggregation-pipeline/#std-label-agg-quick-ref-field-paths), prefix the field name with a dollar sign `$` and enclose it in quotes.<br>Unless [`$bucket`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#mongodb-pipeline-pipe.-bucket) includes a [default](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#std-label-bucket-default) specification, each input document must resolve the `groupBy` field path or expression to a value that falls within one of the ranges specified by the [boundaries.](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#std-label-bucket-boundaries)    |
| [boundaries](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#std-label-bucket-boundaries) | array      | An array of values based on the [groupBy](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#std-label-bucket-group-by) expression that specify the boundaries for each bucket.<br>Each adjacent pair of values acts as the inclusive lower boundary and the exclusive upper boundary for the bucket.<br>You must specify at least two boundaries.<br>The specified values must be in ascending order and all of the same [type](https://www.mongodb.com/docs/manual/reference/bson-types/#std-label-bson-types).<br>The exception is if the values are of mixed numeric types, such as: `[10, NumberLong(20), NumberInt(30)]`<br>For example, an array of `[ 0, 5, 10 ]` creates two buckets:<br><br>- `[0, 5]` with inclusive lower bound `0` and exclusive upper bound `5`.<br>- `[5, 10]` with inclusive lower bound `5` and exclusive upper bound `10`.                          |
| [default](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#std-label-bucket-default)       | literal    | _Optional_.<br>A literal that specifies the `_id` of an additional bucket that contains all documents whose [groupBy](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#std-label-bucket-group-by) expression result does not fall into a bucket specified by [boundaries.](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#std-label-bucket-boundaries)<br>If unspecified, each input document must resolve the `groupBy` expression to a value within one of the bucket ranges specified by `boundaries` or the operation throws an error.<br>The `default` value must be less than the lowest `boundaries` value, or greater than or equal to the highest `boundaries` value.<br>The `default` value can be of a different [type](https://www.mongodb.com/docs/manual/reference/bson-types/#std-label-bson-types) than the entries in `boundaries`. |
| [output](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucket/#std-label-bucket-output)         | document   | _Optional_.<br>A document that specifies the fields to include in the output documents in addition to the `_id` field.<br>To specify the field to include, you must use [accumulator expressions.](https://www.mongodb.com/docs/manual/reference/operator/aggregation/#std-label-agg-quick-reference-accumulators)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |


In [ ]:
db.persons.aggregate([
    { $bucket: { 
        groupBy: "$dob.age", 
        boundaries: [18, 30, 40, 50, 60, 80, 120], // categorias
        output: {
            numPersons: { $sum: 1 },
            averageAge: { $avg: "$dob.age" },
            // names: { 
            //     $push: "$name.first" }
        }
    }}
])

- Output da query acima:

In [ ]:
[
  { _id: 18, numPersons: 868, averageAge: 25.101382488479263 },
  { _id: 30, numPersons: 910, averageAge: 34.51758241758242 },
  { _id: 40, numPersons: 918, averageAge: 44.42265795206972 },
  { _id: 50, numPersons: 976, averageAge: 54.533811475409834 },
  { _id: 60, numPersons: 1328, averageAge: 66.55798192771084 }
]

- Outra possibilidade utilizando `$bucketAuto`:

1. [`$bucketAuto` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/bucketAuto/#-bucketauto--aggregation-)

In [ ]:
db.persons.aggregate([
    { $bucketAuto: {
        groupBy: "$dob.age",
        buckets: 5,
        output: {
            numPersons: { $sum: 1 },
            averageAge: { $avg: "$dob.age" },
            // names: { 
            //     $push: "$name.first" }
        }
    }}
])

- Output da query acima:

In [ ]:
[
  {
    _id: { min: 21, max: 32 },
    numPersons: 1042,
    averageAge: 25.99616122840691
  },
  {
    _id: { min: 32, max: 43 },
    numPersons: 1010,
    averageAge: 36.97722772277228
  },
  {
    _id: { min: 43, max: 54 },
    numPersons: 1033,
    averageAge: 47.98838334946757
  },
  {
    _id: { min: 54, max: 65 },
    numPersons: 1064,
    averageAge: 58.99342105263158
  },
  {
    _id: { min: 65, max: 74 },
    numPersons: 851,
    averageAge: 69.11515863689776
  }
]

---

## 2.21 Diving into additional stages

1. [`$skip` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/skip/#-skip--aggregation-)
2. [`$limit` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/limit/#-limit--aggregation-)

- We wanna find 10 users with the oldest/lowest birthDay
- Then we wanna find the next 10 (like we have a pagination)

In [ ]:
db.persons.aggregate([
    { $project: { 
        _id: 0,
        name: { $concat: ["$name.first", " ", "$name.last"] },
        birthdate: { $toDate: "$dob.date" }
    } },
    { $sort: { birthdate: 1 } },
    { $skip: 10 },
    { $limit: 10 }
])

In [ ]:
db.persons.aggregate([
    { $match: { gender: "male"} },
    { $project: { 
        _id: 0,
        name: { $concat: ["$name.first", " ", "$name.last"] },
        birthdate: { $toDate: "$dob.date" }
    } },
    { $sort: { birthdate: 1 } },
    { $skip: 10 },
    { $limit: 10 }
])

---

## 2.22 How MongoDB optimizes your aggregation pipelines

> MongoDB actually tries its best to optimize your Aggregation Pipelines without interfering with your logic.
>
> Learn more about the default optimizations MongoDB performs in this article: https://docs.mongodb.com/manual/core/aggregation-pipeline-optimization/

---

## 2.23 Writing pipeline results into a new collection

1. [`$out` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/out/#-out--aggregation-)

> `$out`:
> takes the documents returned by the aggregation pipeline and writes them to a specified collection. You can specify the output database.
>
> The $out stage must be the last stage in the pipeline. The $out operator lets the aggregation framework return result sets of any size.

In [ ]:
db.persons.aggregate([
    {
      $project: {
        _id: 0,
        name: 1,
        email: 1,
        birthdate: { $toDate: '$dob.date' },
        age: "$dob.age",
        location: {
          type: 'Point',
          coordinates: [
            {
              $convert: {
                input: '$location.coordinates.longitude',
                to: 'double',
                onError: 0.0,
                onNull: 0.0
              }
            },
            {
              $convert: {
                input: '$location.coordinates.latitude',
                to: 'double',
                onError: 0.0,
                onNull: 0.0
              }
            }
          ]
        }
      }
    },
    {
      $project: {
        gender: 1,
        email: 1,
        location: 1,
        birthdate: 1,
        age: 1,
        fullName: {
          $concat: [
            { $toUpper: { $substrCP: ['$name.first', 0, 1] } },
            {
              $substrCP: [
                '$name.first',
                1,
                { $subtract: [{ $strLenCP: '$name.first' }, 1] }
              ]
            },
            ' ',
            { $toUpper: { $substrCP: ['$name.last', 0, 1] } },
            {
              $substrCP: [
                '$name.last',
                1,
                { $subtract: [{ $strLenCP: '$name.last' }, 1] }
              ]
            }
          ]
        }
      }
    },
    { $out: "transformedPersons" }
  ])

In [ ]:
db.transformedPersons.findOne()

- Output da nova collection criada através da consulta:

In [ ]:
{
    _id: ObjectId('67fd24179c61fed4b9ff1e5d'),
    location: { type: 'Point', coordinates: [ -154.6037, -29.6721 ] },
    email: 'carl.jacobs@example.com',
    birthdate: ISODate('1984-09-30T01:20:26.000Z'),
    age: 33,
    fullName: 'Carl Jacobs'
  }

---

## 2.24 Working with the $geoNear stage

1. [`$geoNear` (aggregation)](https://www.mongodb.com/docs/manual/reference/operator/aggregation/geoNear/#-geonear--aggregation-)

> `$geoNear`: outputs documents in order of nearest to farthest from a specified point.

In [ ]:
db.transformedPersons.createIndex( {location: "2dsphere" }) // location_2dsphere

In [ ]:
db.transformedPersons.aggregate([
    {
        $geoNear: {
            near: { 
                type: "Point", 
                coordinates: [ -18.4, -42.8 ]
            },
            maxDistance: 100000,
            query: { age: { $gt: 30 } },
            distanceField: "distance"
        }
    },
    { $limit: 10 }
])

- Output da query acima:

In [ ]:
[
  {
    _id: ObjectId('67fd24179c61fed4b9ff1e65'),
    location: { type: 'Point', coordinates: [ -18.5996, -42.6128 ] },
    email: 'elijah.lewis@example.com',
    birthdate: ISODate('1986-03-29T06:40:18.000Z'),
    age: 32,
    fullName: 'Elijah Lewis',
    distance: 26473.52536319881
  }
]

---

## 2.25 Wrap Up

<img src="imgs\s12\s12-3.png" width=700 height=250 >

---

## 2.26 Useful resources and links

> Helpful Articles/ Docs:
>
> - Official Aggregation Framework Docs: https://docs.mongodb.com/manual/core/aggregation-pipeline/
> - Learn more about $cond: https://docs.mongodb.com/manual/reference/operator/aggregation/cond/